In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_landsat()
df["p-pet"] = df["precip"] - df["PET"]
df["p-et"] = df["precip"] - df["wp_le"] * .408
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN,p-pet,p-et
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.377924,0.413754,0.594708,31.567899,17.204530,29.105844,24.562860,29.105844,-7.33,-5.566712
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.381184,0.420529,0.597960,29.570000,17.390000,29.760736,26.096551,29.760736,-6.52,-5.507494
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.384444,0.427304,0.601212,31.097908,17.235624,30.299442,25.986418,30.299442,-6.92,-6.115032
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.387704,0.434079,0.604465,30.868718,17.248525,28.978868,25.122209,28.978868,-6.35,-4.633187
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.390963,0.440854,0.607717,30.657792,17.259663,30.653667,27.830367,30.653667,-5.13,-4.464786


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 26 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 non-null float64
wp_LST.night    1028 non-null float64
LW_IN.wp        1028 non-null float64
LW_IN.si        1028 non-null float64
LW_IN           1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2", "wp_RNET", "wp_gpp", "p-pet", "p-et"]
X, Y = exp.featurize(df, train_cols, ["wp_ch4_gf"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 16)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.901226477958
CV Scores:  0.886893486733, 0.925187185905, 0.911905293783, 0.904943864078, 0.90743324266, 0.892069455195, 0.870209629418, 0.897718431633, 0.910829551835, 0.905074638337
OOB score: 0.907598072792
Feature Importances:
('wp_lswi2', 0.2638421110850942)
('wp_evi', 0.17010029958984721)
('wp_ndvi', 0.13077411069354977)
('wp_LST.night', 0.094924975335362802)
('air_temp', 0.056821493052004672)
('wp_gpp', 0.054842109011606278)
('wp_LST.day', 0.045108469098115583)
('wp_RNET', 0.03384977563412557)
('wind_speed', 0.029248160738115505)
('p-et', 0.026060795576417942)
('p-pet', 0.023044886693590994)
('sw_in', 0.022722054839707032)
('VPD', 0.016682214735471036)
('PET', 0.016398818064333819)
('LW_IN.wp', 0.01431645641195043)
('precip', 0.0012632694407071678)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.90290314063
CV Scores:  0.897125026137, 0.922530464759, 0.91269611665, 0.896174742765, 0.912286337297, 0.883972344916, 0.885915733252, 0.891767826041, 0.915226980568, 0.911335833918
Feature Importances:
('wp_lswi2', 0.17264781610841012)
('wp_ndvi', 0.14048681629310941)
('wp_evi', 0.09418131803255099)
('wp_LST.night', 0.08570290374145044)
('wind_speed', 0.067541850845044604)
('wp_gpp', 0.066599812593064953)
('wp_LST.day', 0.061155533557921568)
('sw_in', 0.050035230205214568)
('VPD', 0.045274821135169334)
('LW_IN.wp', 0.040422435834966096)
('air_temp', 0.039312706203261424)
('p-et', 0.038795715437156902)
('wp_RNET', 0.034858524270828591)
('p-pet', 0.030573732277578303)
('PET', 0.028201373471926652)
('precip', 0.0042094099923458807)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.0946671806219
CV Scores:  0.0454689006457, 0.0788591655529, 0.132166698668, 0.0946576332891, -0.0023799788313, 0.167865055238, 0.0838773342286, 0.105354704795, 0.14208909333, 0.0987131993027


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y, regr=r.dnn(nn_steps=10000, nn_lr=0.2, hidden_units=[50, 50]), k=10)

Running Neural Network Cross Validation...
Step #1, avg. loss: 1231880.50000
Step #1001, epoch #100, avg. loss: 353316.15625
Step #2001, epoch #200, avg. loss: 129205.35938
Step #3001, epoch #300, avg. loss: 74146.38281
Step #4001, epoch #400, avg. loss: 46194.24609
Step #5001, epoch #500, avg. loss: 29990.21680
Step #6001, epoch #600, avg. loss: 20172.93164
Step #7001, epoch #700, avg. loss: 14496.40430
Step #8001, epoch #800, avg. loss: 10920.71387
Step #9001, epoch #900, avg. loss: 8656.18652
Step #1, avg. loss: 1267960.50000
Step #1001, epoch #100, avg. loss: 352867.18750
Step #2001, epoch #200, avg. loss: 129861.84375
Step #3001, epoch #300, avg. loss: 73852.88281
Step #4001, epoch #400, avg. loss: 45992.95312
Step #5001, epoch #500, avg. loss: 30313.77148
Step #6001, epoch #600, avg. loss: 21127.19141
Step #7001, epoch #700, avg. loss: 15432.69629
Step #8001, epoch #800, avg. loss: 11832.70801
Step #9001, epoch #900, avg. loss: 9839.35156
Step #1, avg. loss: 1561581.75000
Step #1

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.2,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x118caf9b0>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=10000,
          tf_master='', tf_random_seed=42, verbose=1)